#Libraries

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import xgboost as xgb
from collections import Counter
import numpy as np
import itertools
from pathlib import Path
import pandas as pd
from collections import Counter
from itertools import product
import numpy as np, pandas as pd, re
from sklearn.metrics import f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Data-Sets from myDrive (multiple Fasta Files)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Fasta Files to Dataset Conversion

In [3]:
def load_fasta_file(path: Path):
    seqs, labs = [], []
    lines = path.read_text().splitlines()
    for i in range(0, len(lines), 2):
        header = lines[i].strip()
        if not header:
            continue
        seq  = lines[i+1].strip() if i+1 < len(lines) else ""
        labs.append(1 if "Positive" in header or "POSITIVE" in header or header==">1" else 0)
        seqs.append(seq.upper())
    return seqs, labs

def load_fasta_data(data_dir, files):
    seqs, labs, per_file_counts = [], [], {}
    for fname in files:
        seq, lab = load_fasta_file(Path(data_dir) / fname)
        seqs.extend(seq)
        labs.extend(lab)
        per_file_counts[fname] = Counter(lab)

    lengths = [len(s) for s in seqs]
    min_len = min(lengths)
    max_len = max(lengths)

    df = pd.DataFrame({"seq": seqs, "label": labs})

    print(f"Shortest sequence length in original dataset : {min_len}")
    print(f"Longest  sequence length in original dataset : {max_len}")

    print(f"\nTotal sequences : {len(df):,}")
    print(f"Class balance   : {Counter(df.label)}")

    print("\nBreak-down per file:")
    for f, c in per_file_counts.items():
        print(f"  {f:<12}  Pos={c[1]:>4}  Neg={c[0]:>4}")

    print("\nDataFrame preview:")
    display(df.sample(5))
    print("\n\n")
    return df

# Load data for each species using the new function
data_dir_fruit_fly = "/content/drive/MyDrive/Tri-Ori/training"
files_fruit_fly = ["bmark-BG3_F_F.txt", "bmark-KC_F_F.txt", "bmark-S2_F_F.txt"]
df_fruit_fly = load_fasta_data(data_dir_fruit_fly, files_fruit_fly) # df_fruit_fly = load_fasta_data(ff_dir, ff_files)

data_dir_human = "/content/drive/MyDrive/Tri-Ori/training"
files_human = ["bmark-K562_Human.txt", "bmark-MCF7_Human.txt"]
df_human = load_fasta_data(data_dir_human, files_human)

print('eije shuru',df_human)

data_dir_mouse = "/content/drive/MyDrive/Tri-Ori/training"
files_mouse = ["bmark-ES_Mouse.txt", "bmark-P19_Mouse.txt", "bmark-MEF_Mouse.txt"]
df_mouse = load_fasta_data(data_dir_mouse, files_mouse)

Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 18,250
Class balance   : Counter({1: 9135, 0: 9115})

Break-down per file:
  bmark-BG3_F_F.txt  Pos=2763  Neg=2764
  bmark-KC_F_F.txt  Pos=3022  Neg=3000
  bmark-S2_F_F.txt  Pos=3350  Neg=3351

DataFrame preview:


,seq,label
14978,TTAATAGTGAGCAAACGAAATAATTTATTTTATTTAAATAAAATAG...,0
1226,TCAGCTTCAACCTCTGGACCATTCCCAACTTCTTGGTGCAGTGCCT...,1
9093,ACACACATTTTTAATGGTCACAATCACATCGTCCATGTATACGACT...,0
16824,AAAGCCGACTGCACCGGAACGACCAACCAGCCCAGCGTTAAATTTG...,0
6,TGCAGTCGAATTCTCTGCCTCTGGCGCTGCTCGCCAACTTATGTGA...,1





Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 6,189
Class balance   : Counter({1: 3095, 0: 3094})

Break-down per file:
  bmark-K562_Human.txt  Pos=1332  Neg=1331
  bmark-MCF7_Human.txt  Pos=1763  Neg=1763

DataFrame preview:


,seq,label
5943,GAGAGTCTTTTAGGTGAGGCCGCTGTGTCACCTGCTTGGGGTCGCA...,0
4455,CCCGTGGGCTGCAGTTGGAACGATGGCGGCGGCAGCTGCCGCCGGG...,0
1919,CTCGTGCCTGGGGAATGGGGATCATGGGTGCTGGCATTGTGGGTAG...,0
5689,CTTGACTTTCCTCCCTGTGTATCCCTGCGCCAGCAGTTCATCTCTC...,0
4982,GATTGGCCAGCTTGCTCTGTGCCACTTGTTTTATTTATTTGTTTGC...,0





eije shuru                                                     seq  label
0     TGGAGGTACCTGCTCTCCACGCAGTACAGCATCTGGCTGTGTGGTG...      1
1     CAGCCCAGCCCCTCCAGGGCTCTGAGAGAGTTGTCTCCCAGGCCCA...      1
2     CCGGTCCGAAGGGCAAATAGTCTCGGGTCTCTCCGATTTCCCGAAC...      1
3     AGAGGGAGGGGCCGACGTGTGGGAGAGGGAAGGAAAGGGGGAGGAG...      1
4     GCTCCGGCCGCTGACGCCAAAATAGGGAAGGCCGGAAGCGGCGGGG...      1
...                                                 ...    ...
6184  CAGTTGCGCTTGAGGTGTCCGTAGAAGCTGGAATCGAGCCCTGCGG...      0
6185  CTTAACACGTTAAAGAGTCCAATTGGAGAGCAGATTATTTAAGGTG...      0
6186  GTCCCAGCCGGACGGTGGCCTCTTGTGTTCAGGGACAAGGCAAGGA...      0
6187  CTCCTGTGAGCAGAGCAGAGCAGACCCGTCAGGCACCACCGGCCTC...      0
6188  TAAGGGATCCAAGAGGAATAGAGGGTGAAGGAAATGGCCATGAAGG...      0

[6189 rows x 2 columns]
Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 8,614
Class balance   : Counter({1: 4307, 0: 4307})

Break-down per file:
  bmark-ES

,seq,label
1536,CTTGAATTGTTTTTCTCCGACTCCTGAAACCAGTGTATCCTGAGGA...,0
3789,GGCCTGGGGCACAGCGGCAGTAGGAAGGCCCAGGGGACCTCCGAAG...,1
2384,GGAGAGCACATAAGCAGATGAAGGGGAGGATGTCAAAGTCCTTTAT...,0
3545,TGGGATCCCGGTGTGAGGGGTGCTGGCTAAGCCCTCCTGTGTGAGG...,1
7376,TGTGGGTTGCAGAAGTCAGAGGACTGTTTGCAAGAGTGGGTTTCAT...,1


## Encoding (Embedding): K-mer Counts

In [4]:
def count_kmers(sequence, k):
    kmer_counts = Counter()
    num_kmers = len(sequence) - k + 1
    for i in range(num_kmers):
        kmer = sequence[i:i+k]
        kmer_counts[kmer] += 1
    return kmer_counts

k_size = 5

df_fruit_fly_kmer = pd.DataFrame({
    'label': df_fruit_fly['label'],
    'kmer_counts': df_fruit_fly['seq'].apply(lambda seq: count_kmers(seq, k_size))
})

df_human_kmer = pd.DataFrame({
    'label': df_human['label'],
    'kmer_counts': df_human['seq'].apply(lambda seq: count_kmers(seq, k_size))
})

df_mouse_kmer = pd.DataFrame({
    'label': df_mouse['label'],
    'kmer_counts': df_mouse['seq'].apply(lambda seq: count_kmers(seq, k_size))
})


print(f"Fruit Fly DataFrame with {k_size}-mer counts:")
display(df_fruit_fly_kmer.head(2))

print(f"\nHuman DataFrame with {k_size}-mer counts:")
display(df_human_kmer.head(2))

print(f"\nMouse DataFrame with {k_size}-mer counts:")
display(df_mouse_kmer.head(2))

Fruit Fly DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'GATAT': 2, 'ATATT': 6, 'TATTT': 3, 'ATTTA': ..."
1,1,"{'GATGA': 1, 'ATGAC': 1, 'TGACT': 1, 'GACTC': ..."



Human DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'TGGAG': 3, 'GGAGG': 8, 'GAGGT': 2, 'AGGTA': ..."
1,1,"{'CAGCC': 3, 'AGCCC': 3, 'GCCCA': 3, 'CCCAG': ..."



Mouse DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'CCCCT': 2, 'CCCTA': 1, 'CCTAA': 1, 'CTAAC': ..."
1,1,"{'GGGCA': 1, 'GGCAA': 1, 'GCAAT': 1, 'CAATC': ..."


# **Log Reg Scores on training_dataset (80:20)**

In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# ---------- Helper function ----------
def run_logistic_regression(df_kmer, species_name):
    # Convert dictionary of kmer_counts into feature vectors
    vec = DictVectorizer(sparse=False)
    X = vec.fit_transform(df_kmer['kmer_counts'])
    y = df_kmer['label'].values

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Logistic Regression model
    clf = LogisticRegression(max_iter=500)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Metrics
    acc = accuracy_score(y_test, y_pred)
    macro_f1 = f1_score(y_test, y_pred, average="macro")
    pos_f1 = f1_score(y_test, y_pred, pos_label=1)
    neg_f1 = f1_score(y_test, y_pred, pos_label=0)

    return {
        "Model": "Logistic Regression",
        "Embedding": f"{k_size}-mer Freq",
        "Species": species_name,
        "Test Acc": round(acc, 4),
        "Test macro avg f1": round(macro_f1, 4),
        "Positive f1": round(pos_f1, 4),
        "Negative f1": round(neg_f1, 4)
    }

# ---------- Run for all three species ----------
results = []
results.append(run_logistic_regression(df_fruit_fly_kmer, "Fruit Fly"))
results.append(run_logistic_regression(df_human_kmer, "Human"))
results.append(run_logistic_regression(df_mouse_kmer, "Mouse"))

# Put into a DataFrame for a nice table
results_df = pd.DataFrame(results)
print(results_df)


                 Model   Embedding    Species  Test Acc  Test macro avg f1  \
0  Logistic Regression  5-mer Freq  Fruit Fly    0.7384             0.7384   
1  Logistic Regression  5-mer Freq      Human    0.8053             0.8053   
2  Logistic Regression  5-mer Freq      Mouse    0.8091             0.8090   

   Positive f1  Negative f1  
0       0.7373       0.7394  
1       0.8036       0.8070  
2       0.8059       0.8121  


# **Loading Testing Data Set**

In [6]:
def load_fasta_file(path: Path):
    seqs, labs = [], []
    lines = path.read_text().splitlines()
    for i in range(0, len(lines), 2):
        header = lines[i].strip()
        if not header:
            continue
        seq  = lines[i+1].strip() if i+1 < len(lines) else ""
        labs.append(1 if "Positive" in header or "POSITIVE" in header or header==">1" else 0)
        seqs.append(seq.upper())
    return seqs, labs

def load_fasta_data(data_dir, files):
    seqs, labs, per_file_counts = [], [], {}
    for fname in files:
        seq, lab = load_fasta_file(Path(data_dir) / fname)
        seqs.extend(seq)
        labs.extend(lab)
        per_file_counts[fname] = Counter(lab)

    lengths = [len(s) for s in seqs]
    min_len = min(lengths)
    max_len = max(lengths)

    df = pd.DataFrame({"seq": seqs, "label": labs})

    print(f"Shortest sequence length in original dataset : {min_len}")
    print(f"Longest  sequence length in original dataset : {max_len}")

    print(f"\nTotal sequences : {len(df):,}")
    print(f"Class balance   : {Counter(df.label)}")

    print("\nBreak-down per file:")
    for f, c in per_file_counts.items():
        print(f"  {f:<12}  Pos={c[1]:>4}  Neg={c[0]:>4}")

    print("\nDataFrame preview:")
    display(df.sample(5))
    print("\n\n")
    return df

# Load data for each species using the new function
data_dir_fruit_fly = "/content/drive/MyDrive/Tri-Ori/independent"
files_fruit_fly = ["Ind-Bg3.txt", "Ind-KC.txt", "Ind-S2.txt"]
df_test_fruit_fly = load_fasta_data(data_dir_fruit_fly, files_fruit_fly) # df_fruit_fly = load_fasta_data(ff_dir, ff_files)

data_dir_human = "/content/drive/MyDrive/Tri-Ori/independent"
files_human = ["Ind-K562.txt", "Ind-MCF7.txt"]
df_test_human = load_fasta_data(data_dir_human, files_human)

print('eije shuru',df_human)

data_dir_mouse = "/content/drive/MyDrive/Tri-Ori/independent"
files_mouse = ["Ind-ES.txt", "Ind-P19.txt", "Ind-MEF.txt"]
df_test_mouse = load_fasta_data(data_dir_mouse, files_mouse)

Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 16,000
Class balance   : Counter({1: 8000, 0: 8000})

Break-down per file:
  Ind-Bg3.txt   Pos=2000  Neg=2000
  Ind-KC.txt    Pos=3000  Neg=3000
  Ind-S2.txt    Pos=3000  Neg=3000

DataFrame preview:


,seq,label
7039,ATGATGTTTTAGCTGGAGCGGACTCCACGGAAGAAGCTCAGCTCAT...,0
11255,ATATGAATGACTGTATATGGGAGAGGGCGAAAAGAAATTGCCAGCA...,1
8371,GCTGCCTTGCGATCGTTCCTTCGATGACTGATTGTCGATAACTTCG...,0
7155,CTTGGTTGCATTAGCAAAGGGATATCTCTTTTTAGGCTTTCAGCAT...,0
13748,AAATTGGAATTCAAAATTACTGAACTTACTACTAATTTCTTACTAC...,0





Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 4,000
Class balance   : Counter({1: 2000, 0: 2000})

Break-down per file:
  Ind-K562.txt  Pos=1000  Neg=1000
  Ind-MCF7.txt  Pos=1000  Neg=1000

DataFrame preview:


,seq,label
57,ACCTGCTTTGGCCCCAAAATTTCAGAAAAGCCCCGAGAAGGAAGAA...,1
2341,GTCCTCCCTGCAGGCCCCGCAGGAGGGGCAGAGACACCGAGGCTGC...,1
377,TGTTCCGGGAGGACCCCAGTCTTTGTCCGTGACTAGAGGAGACGAA...,1
1001,TTTGTGTAGAAGAGGCGCTGGGACCCTTGAGGTTGGAGGCTCAACA...,0
2612,GCACCGTCCCGAGCTTAGTGTCCGGTTCTTACAATGGGTCGTCGAG...,1





eije shuru                                                     seq  label
0     TGGAGGTACCTGCTCTCCACGCAGTACAGCATCTGGCTGTGTGGTG...      1
1     CAGCCCAGCCCCTCCAGGGCTCTGAGAGAGTTGTCTCCCAGGCCCA...      1
2     CCGGTCCGAAGGGCAAATAGTCTCGGGTCTCTCCGATTTCCCGAAC...      1
3     AGAGGGAGGGGCCGACGTGTGGGAGAGGGAAGGAAAGGGGGAGGAG...      1
4     GCTCCGGCCGCTGACGCCAAAATAGGGAAGGCCGGAAGCGGCGGGG...      1
...                                                 ...    ...
6184  CAGTTGCGCTTGAGGTGTCCGTAGAAGCTGGAATCGAGCCCTGCGG...      0
6185  CTTAACACGTTAAAGAGTCCAATTGGAGAGCAGATTATTTAAGGTG...      0
6186  GTCCCAGCCGGACGGTGGCCTCTTGTGTTCAGGGACAAGGCAAGGA...      0
6187  CTCCTGTGAGCAGAGCAGAGCAGACCCGTCAGGCACCACCGGCCTC...      0
6188  TAAGGGATCCAAGAGGAATAGAGGGTGAAGGAAATGGCCATGAAGG...      0

[6189 rows x 2 columns]
Shortest sequence length in original dataset : 300
Longest  sequence length in original dataset : 300

Total sequences : 6,000
Class balance   : Counter({1: 3000, 0: 3000})

Break-down per file:
  Ind-ES.t

,seq,label
4011,CAGGGTGTTGAATGGGTGGGTGGATGGGTGGATGGGTGGATGGGTG...,1
5377,GACAGATACAGTGCACTTAGATATAATAATAAATAAATCTTTAAAA...,0
2582,AGCTACCAAACACTGTGCCAATGCCAGCTCCTGATCCAGCCACACC...,1
533,ACGCGGGCAGGTCCTCGTGGGCGGGGGGCTGGGGAAGATTGCAAAG...,1
2172,GCATCCTTGAGTGTGCTCTTAAATGCACAGCAGCCCCTCCAGCCCG...,1


# embedding testing set (kmer freq)

In [7]:
def count_kmers(sequence, k):
    kmer_counts = Counter()
    num_kmers = len(sequence) - k + 1
    for i in range(num_kmers):
        kmer = sequence[i:i+k]
        kmer_counts[kmer] += 1
    return kmer_counts

k_size = 5

df_test_fruit_fly_kmer = pd.DataFrame({
    'label': df_test_fruit_fly['label'],
    'kmer_counts': df_test_fruit_fly['seq'].apply(lambda seq: count_kmers(seq, k_size))
})

df_test_human_kmer = pd.DataFrame({
    'label': df_test_human['label'],
    'kmer_counts': df_test_human['seq'].apply(lambda seq: count_kmers(seq, k_size))
})

df_test_mouse_kmer = pd.DataFrame({
    'label': df_test_mouse['label'],
    'kmer_counts': df_test_mouse['seq'].apply(lambda seq: count_kmers(seq, k_size))
})


print(f"Fruit Fly DataFrame with {k_size}-mer counts:")
display(df_test_fruit_fly_kmer.head(2))

print(f"\nHuman DataFrame with {k_size}-mer counts:")
display(df_test_human_kmer.head(2))

print(f"\nMouse DataFrame with {k_size}-mer counts:")
display(df_test_mouse_kmer.head(2))

Fruit Fly DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'GCCGA': 1, 'CCGAG': 1, 'CGAGT': 2, 'GAGTT': ..."
1,1,"{'TGTGG': 2, 'GTGGC': 2, 'TGGCC': 2, 'GGCCA': ..."



Human DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'TCTGT': 1, 'CTGTC': 2, 'TGTCT': 2, 'GTCTT': ..."
1,1,"{'GGGCC': 1, 'GGCCC': 1, 'GCCCC': 1, 'CCCCA': ..."



Mouse DataFrame with 5-mer counts:


,label,kmer_counts
0,1,"{'GGTGT': 2, 'GTGTG': 2, 'TGTGT': 2, 'GTGTT': ..."
1,1,"{'GTGAC': 2, 'TGACC': 2, 'GACCC': 3, 'ACCCC': ..."


# LogReg {training_df: initial_df, testing_df: independant_df}

In [8]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# --------- Helper function ---------
def cross_check_logreg(train_df, test_df, species_name):
    # Vectorize k-mer counts
    vec = DictVectorizer(sparse=False)
    X_train = vec.fit_transform(train_df['kmer_counts'])
    y_train = train_df['label'].values

    # Important: transform test data with same vectorizer
    X_test = vec.transform(test_df['kmer_counts'])
    y_test = test_df['label'].values

    # Train Logistic Regression
    clf = LogisticRegression(max_iter=500)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Metrics
    macro_f1 = f1_score(y_test, y_pred, average="macro")
    pos_f1 = f1_score(y_test, y_pred, pos_label=1)
    neg_f1 = f1_score(y_test, y_pred, pos_label=0)

    return {
        "Species": species_name,
        "Cross Test macro avg f1": round(macro_f1, 4),
        "Cross Test Positive f1": round(pos_f1, 4),
        "Cross Test Negative f1": round(neg_f1, 4)
    }

# --------- Run for each species ---------
results_cross = []
results_cross.append(cross_check_logreg(df_fruit_fly_kmer, df_test_fruit_fly_kmer, "Fruit Fly"))
results_cross.append(cross_check_logreg(df_human_kmer, df_test_human_kmer, "Human"))
results_cross.append(cross_check_logreg(df_mouse_kmer, df_test_mouse_kmer, "Mouse"))

# Put results in DataFrame
cross_df = pd.DataFrame(results_cross)
print(cross_df)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


     Species  Cross Test macro avg f1  Cross Test Positive f1  \
0  Fruit Fly                   0.7308                  0.7308   
1      Human                   0.8052                  0.8057   
2      Mouse                   0.8182                  0.8177   

   Cross Test Negative f1  
0                  0.7308  
1                  0.8048  
2                  0.8186  


# **Cross Check**

In [9]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def cross_check_logreg(train_df, test_df):
    # Vectorize k-mer counts
    vec = DictVectorizer(sparse=False)
    X_train = vec.fit_transform(train_df['kmer_counts'])
    y_train = train_df['label'].values

    X_test = vec.transform(test_df['kmer_counts'])
    y_test = test_df['label'].values

    # Train Logistic Regression
    clf = LogisticRegression(max_iter=500)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    # Return macro F1 only (can expand if needed)
    return round(f1_score(y_test, y_pred, average="macro"), 4)

# Datasets dictionary
datasets_train = {
    "Human": df_human_kmer,
    "Mouse": df_mouse_kmer,
    "Fruit Fly": df_fruit_fly_kmer
}

datasets_test = {
    "Human": df_test_human_kmer,
    "Mouse": df_test_mouse_kmer,
    "Fruit Fly": df_test_fruit_fly_kmer
}

# Build results matrix
results_matrix = {}
for train_species, train_df in datasets_train.items():
    row_results = {}
    for test_species, test_df in datasets_test.items():
        row_results[f"{test_species} Cross Test f1"] = cross_check_logreg(train_df, test_df)
    results_matrix[train_species] = row_results

# Convert to DataFrame
cross_matrix_df = pd.DataFrame(results_matrix).T
print(cross_matrix_df)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

           Human Cross Test f1  Mouse Cross Test f1  Fruit Fly Cross Test f1
Human                   0.8052               0.5471                   0.3912
Mouse                   0.5940               0.8182                   0.4056
Fruit Fly               0.4156               0.5922                   0.7308


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
